# Imports

In [ ]:
import numpy as np
import pandas as pd
import requests
from io import BytesIO, StringIO
import os

## Download and prepare a labelled dataset with the url of the images

In [ ]:
url = "http://world.openfoodfacts.org/data/en.openfoodfacts.org.products.csv"
s = requests.get(url).content
whole_data = pd.read_csv(StringIO(s.decode('utf-8')), sep='\t')

#Keep two brands only: Carrefour = 1 and Auchan = 0
two_classes_data = whole_data.query("brands == 'Carrefour'|brands == 'Auchan'")
two_classes_data = two_classes_data.reset_index()
two_classes_data = two_classes_data.loc[:,['brands','image_url',
                                           #'image_small_url'
                                          ]
                                       ]

#two_classes_data.brands.value_counts()
two_classes_data.brands = two_classes_data.brands.str.replace('Carrefour', '1')
two_classes_data.brands = two_classes_data.brands.str.replace('Auchan', '0')

two_classes_data = two_classes_data.loc[two_classes_data.image_url.notnull(),:]
two_classes_data = two_classes_data.reset_index()
del two_classes_data['index']

two_classes_data['image_name'] = two_classes_data.image_url.str[44:].str.replace('/','')
labels = two_classes_data.loc[:,['brands','image_name']].set_index('image_name')
labels.rename(columns = {'brands':'label'}, inplace = True)
labels.to_csv(os.path.join(wd_path,'labels.csv'))

## Define an auxiliary function to download the images from their url

In [2]:
path = '/home/kevin/Desktop/OpenFoodFacts/OpenFoodFacts/data/images'

def download_url(image_url, image_name, directory_path = path):
    """
        Downloads the images from url into selected directory
        
        Arguments:
            *url: the url of the site
            *image_name: the name to use to save the image
            *directory_path: the path of the directory where to save
        
        Returns:
        
    """
    r = requests.get(image_url)
    pic = Image.open(BytesIO(r.content))
    pic.save(os.path.join(path, image_name))
    print("Saving the image {} in working directory \n".format(image_name))


## Download the images in a directory

In [ ]:
two_classes_data.apply(lambda x: download_url(x['image_url'], x['image_name']), 
                       axis = 1)
 

## Load them into a numpy array

See the script 
```
read.py
```

In [3]:
#TODO: il manque les labels